In [1]:
import os, sys
import pandas as pd
import numpy as np
import re
import resource
import cplex

In [2]:
from cobra.io import read_sbml_model, write_sbml_model
from cobra.util.solver import solvers
from cobra.core.configuration import Configuration

In [3]:
 Configuration.solver = solvers['cplex']

In [4]:
from cobra.flux_analysis import find_blocked_reactions
from urllib.request import urlretrieve
from cobamp.utilities.parallel import batch_run, cpu_count
from troppo.methods_wrappers import ReconstructionWrapper
from troppo.omics.core import OmicsMeasurementSet, OmicsContainer

from troppo.methods_wrappers import integration_strategy_map
from troppo.omics.integration import MINSUM, MINMAX

from itertools import product

/home/jiang817/anaconda3/envs/BaloniLab_py38/lib/python3.8/site-packages/cobamp/wrappers/external_wrappers.py:9: UserWarning: 
The wrappers.external_wrappers module will be deprecated in a future release in favour of the wrappers module. 
    Available ModelObjectReader classes can still be loaded using cobamp.wrappers.<class>. An appropriate model 
    reader can also be created using the get_model_reader function on cobamp.wrappers
  warnings.warn(


In [5]:
arg = [argval.split('=')[1] for argval in sys.argv[1:] if '-config=' in argval]
mp = '-no-mp' not in sys.argv[1:]

In [47]:
# mmodify Recon3D model
model = read_sbml_model('/home/jiang817/cybergut_project/General_models/Recon3D.xml')
model_cobra = read_sbml_model('/home/jiang817/cybergut_project/General_models/Recon3D.xml')

In [ ]:
# model.remove_metabolites([m for m in model.metabolites if m.compartment == 'x'])


In [7]:
blocked = find_blocked_reactions(model)


In [8]:
model.remove_reactions(blocked)

In [7]:
# model.reactions.get_by_id('BIOMASS_maintenance').bounds = (0, 1000)
for i in range(len(model.reactions)):
    if model.reactions[i].id.startswith('EX_'):
        model.reactions[i].bounds = (-100, 100)


In [28]:
exp_data = pd.read_csv('/home/jiang817/cybergut_project/Draft_model/Recon3D_input/Recon3D_XML_tINIT_INPUT/expressiondata_tinit_log2.csv')

In [29]:
exp_data

,EntrezID,score
0,53947_AT1,0.000000
1,8086_AT1,10.807681
2,65985_AT1,10.209345
3,60496_AT1,10.289423
4,10157_AT1,0.000000
...,...,...
1659,51304_AT1,11.677349
1660,55146_AT1,0.000000
1661,64429_AT1,10.311369
1662,55625_AT1,11.600129


In [30]:
data_dict = {}

In [31]:
exp_data.iloc[1,0]

'8086_AT1'

In [32]:
for i in range(len(exp_data)):
    data_dict[exp_data.iloc[i, 0]] = exp_data.iloc[i, 1]

In [33]:
data_dict

{'53947_AT1': 0.0,
 '8086_AT1': 10.80768098,
 '65985_AT1': 10.2093452,
 '60496_AT1': 10.28942337,
 '10157_AT1': 0.0,
 '18_AT1': 10.80847916,
 '19_AT1': 10.35567385,
 '21_AT1': 10.6839766,
 '5243_AT1': 0.0,
 '10058_AT1': 0.0,
 '4363_AT1': 10.81957497,
 '1244_AT1': 0.0,
 '8714_AT1': 0.0,
 '10257_AT1': 0.0,
 '10057_AT1': 12.07935767,
 '215_AT1': 0.0,
 '5825_AT1': 10.83026326,
 '9619_AT1': 10.05104664,
 '9429_AT1': 0.0,
 '51099_AT1': 0.0,
 '57406_AT1': 0.0,
 '25841_AT1': 9.67782352,
 '30_AT1': 11.39618523,
 '10449_AT1': 11.8535092,
 '31_AT1': 11.19317408,
 '32_AT1': 11.41542947,
 '80724_AT1': 10.09276116,
 '27034_AT1': 10.58651549,
 '34_AT1': 11.18940977,
 '35_AT1': 10.7989507,
 '36_AT1': 10.77313021,
 '37_AT1': 14.28078193,
 '38_AT1': 11.3292505,
 '39_AT1': 0.0,
 '1636_AT1': 0.0,
 '43_AT1': 0.0,
 '47_AT1': 11.99565943,
 '48_AT1': 10.98931021,
 '50_AT1': 12.61731809,
 '26027_AT1': 0.0,
 '55856_AT1': 9.654995393,
 '11332_AT1': 10.11604279,
 '10005_AT1': 9.708730191,
 '23597_AT1': 10.5178502

In [34]:
protected = ['BIOMASS_maintenance', 'ATPS4mi']

In [35]:
#### parameter setup
params = {
            'algorithms':['tinit','fastcore'],
            'strategies': {
                'tinit': integration_strategy_map['adjusted_score'](protected),
                'fastcore': integration_strategy_map['default_core'](0, protected)
            },
            'functions':{'minmax': MINMAX, 'minsum': MINSUM},
    }

In [115]:
 def reconstruct_model(options, params):
    print('\tResource statistics before reconstruction with',options[0],':', resource.getrusage(resource.RUSAGE_SELF))
    alg, d, func = options
    data_dict, aofunc = d, params['functions'][func]

    oc_sample = OmicsContainer(omicstype='transcriptomics', condition='x',
                                data=data_dict, nomenclature='custom')

    return params['rw'].run_from_omics(omics_data=oc_sample, algorithm=alg, and_or_funcs=aofunc,
                        integration_strategy=params['strategies'][alg], solver='GUROBI', raise_errors=False)


In [37]:
# obtain S, ub and lb of Recon3D
rw = ReconstructionWrapper(model, ttg_ratio=9999)
params['rw'] = rw

/home/jiang817/anaconda3/envs/BaloniLab_py38/lib/python3.8/site-packages/cobamp/gpr/core.py:41: UserWarning: Could not normalize this rule: _1591_AT1
  warnings.warn('Could not normalize this rule: ' + rule)
/home/jiang817/anaconda3/envs/BaloniLab_py38/lib/python3.8/site-packages/cobamp/gpr/core.py:41: UserWarning: Could not normalize this rule: _8639_AT1 or _26_AT1 or _314_AT2 or _314_AT1
  warnings.warn('Could not normalize this rule: ' + rule)
/home/jiang817/anaconda3/envs/BaloniLab_py38/lib/python3.8/site-packages/cobamp/gpr/core.py:41: UserWarning: Could not normalize this rule: _1594_AT1
  warnings.warn('Could not normalize this rule: ' + rule)
/home/jiang817/anaconda3/envs/BaloniLab_py38/lib/python3.8/site-packages/cobamp/gpr/core.py:41: UserWarning: Could not normalize this rule: _10993_AT1
  warnings.warn('Could not normalize this rule: ' + rule)
/home/jiang817/anaconda3/envs/BaloniLab_py38/lib/python3.8/site-packages/cobamp/gpr/core.py:41: UserWarning: Could not normalize thi

In [144]:
x = rw.run_from_omics

<bound method ReconstructionWrapper.run_from_omics of <troppo.methods_wrappers.ReconstructionWrapper object at 0x14599482e250>>

In [44]:
algo, intfx  = ('tinit', 'minmax')
#algo, intfx  = ('tinit', 'minsum')

In [116]:
reconstructions = reconstruct_model((algo, data_dict, intfx), params)

	Resource statistics before reconstruction with tinit : resource.struct_rusage(ru_utime=233.234253, ru_stime=44.223327, ru_maxrss=4607244, ru_ixrss=0, ru_idrss=0, ru_isrss=0, ru_minflt=3521833, ru_majflt=433, ru_nswap=0, ru_inblock=101440, ru_oublock=303008, ru_msgsnd=0, ru_msgrcv=0, ru_nsignals=0, ru_nvcsw=403584, ru_nivcsw=8067)
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 921650
Academic license - for non-commercial use only - registered to jiang817@purdue.edu
Set parameter OutputFlag to value 1
Discarded solution information
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD EPYC 7763 64-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 128 physical cores, 256 logical processors, using up to 32 threads


Consider calling update less frequently.

Academic license - for non-commercial use only - registered to jiang817@purdue.edu
Optimize a model with 21704 rows, 53442 columns and 107657 nonzeros
Model fingerpri

In [147]:
reconstructions.values()

dict_values([False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, True, False, False, False, False, False, True, True, True, True, False, False, True, True, False, True, True, False, True, True, True, True, False, False, True, True, False, True, False, True, True, True, True, False, True, True, False, True, False, True, True, True, False, True, False, False, True, False, True, True, True, True, True, True, False, False, True, True, True, True, True, True, True, False, True, True, False, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, False, False, True, False, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True,

In [148]:
num_remove = 0
for i in reconstructions.values():
    if i == False:
        num_remove += 1
print(num_remove)

2742


In [149]:
rm_rxn = []
for (k, v) in reconstructions.items():
    if v == False:
        print(k)
        rm_rxn.append(k)



24_25DHVITD3tm
25HVITD3t
13DAMPPOX
24_25DHVITD2t
24_25DHVITD3t
25VITD2Hm
2AMACHYD
2AMACSULT
2AMADPTm
34DHXMANDACOX
34HPLFM
3MOBt2im
3SALAOX
4HOXPACDOX_NADP
4MOPt2im
4PYRDX
5ADTSTSTERONESULT
5THFtm
A4GALTc
A4GNT2g
ABO5g
ABO7g
ABO9g
ABTt
ACALDtx
ACCOALm
ACGAMtly
ACHVESSEC_1
ACODA
ADNt4
ADNtl
ADPMAN
AGTim
ALOX15
ARGt4
ARTCOAL3_cho
R_group_phosphotase_2
ASCBOX_1
ASNNm
B3GALT42g
B3GNT39g
BAAT1x
BALAVECSEC_1
BETALDHxm
BHBtm
BHMT
BTNDm
BZtr
C3STDH1r
C4STMO2Pr
CH25H
CHLP
CHOLPtl
CLPNDCOAtx
CMPACNAtn
COQ3m
CORE7GTg
CRNCAR3tp
CRTNsyn_cho
CSPG_Ct
CSPG_Dt
CYSTGLUex
CYTD
CYTDn
CYTDtl
DADNK
DARGOp
DASPO1p
DCYTD
DCYTDn
DESAT22_2p
DHEASULT
DHORD9
DM_core5_g
DM_dem2emgacpail_prot_hs_r
DM_dgpi_prot_hs_r
DM_dsT_antigen_g
DM_kdn_c
DM_melanin_c
DM_n5m2masn_g
DM_oretn_n
DM_Ser_Thr_l
DM_sprm_c
DMGDHm
DMHPTCRNCPT2
DNDPt10m
DNDPt11m
DNDPt13m
DNDPt15m
DNDPt16m
DNDPt17m
DNDPt18m
DNDPt1m
DNDPt20m
DNDPt21m
DNDPt25m
DNDPt28m
DNDPt37m
DNDPt38m
DNDPt39m
DNDPt40m
DNDPt42m
DNDPt43m
DNDPt45m
DNDPt49m
DNDPt50m
DNDPt52m
D

In [154]:
len(rm_rxn)


2742

In [165]:
rxn_removed = [[i] for i in rm_rxn]

In [168]:
name = ['rxn_removed']
rxn_removed = pd.DataFrame(columns = name, data = rxn_removed)
rxn_removed.to_csv('/home/jiang817/cybergut_project/Draft_model/Draft_model_with_tINIT/rxn_removed.csv')

In [150]:
model_tINIT  = model_cobra

In [151]:
model_tINIT.remove_reactions(rm_rxn, True)

In [153]:
model_tINIT.optimize()

,fluxes,reduced_costs


In [152]:
write_sbml_model(model_tINIT, "/home/jiang817/cybergut_project/Draft_model/Draft_model_with_tINIT/draft_model_tINIT.xml")

In [3]:
model = read_sbml_model('/home/jiang817/cybergut_project/Draft_model/Draft_model_with_tINIT/draft_model_tINIT_BIGG.xml')

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 921650
Academic license - for non-commercial use only - registered to jiang817@purdue.edu


In [4]:
model

Name,Recon3D
Memory address,2abda0c7a7c0
Number of metabolites,5811
Number of reactions,7897
Number of genes,2039
Number of groups,0
Objective expression,1.0*BIOMASS_maintenance - 1.0*BIOMASS_maintenance_reverse_5b3f9
Compartments,"cytosol, lysosome, mitochondria, endoplasmic reticulum, extracellular space, peroxisome/glyoxysome, nucleus, golgi apparatus, inner mitochondrial compartment"
